# Inferenza del Significato da Definizioni utilizzando WordNet

## Introduzione:
L'esercizio consiste nell'implementare un algoritmo in grado di inferire il concetto descritto da un insieme di definizioni associate ad esso. L'algoritmo che abbiamo sviluppato può essere riassunto nelle seguenti fasi:

1. Caricamento del dataset delle definizioni.
2. Preelaborazione: tokenizzazione, conversione in minuscolo, lemmatizzazione, rimozione delle stopwords, rimozione della punteggiatura, rimozione dei token non presenti in WordNet, 
3. Disambiguazione (algoritmo Lesk) e conteggio delle frequenze dei sensi disambiguati.
4. Esecuzione dell'algoritmo di esplorazione dei sensi di WordNet:
   - (a) Scelta dei candidati "genus" (selezionando i sensi più frequenti).
   - (b) Per ogni candidato "genus", esecuzione di una ricerca in profondità a partire dal sotto-albero del senso "genus" che massimizza la similarità con le definizioni.
   - (c) Scelta del senso con la massima similarità tra quelli estratti dai vari sotto-alberi dei candidati "genus".




Librerie e corpus

In [8]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from statistics import mean
from itertools import product, starmap
from nltk.corpus import wordnet as wn
from collections import Counter

# Downloading necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Data Loading

Caricamento del dataset di definizioni relative ai concetti usando la libreria pandas


In [9]:
def load_dataset(file_path):

    try:
        data = pd.read_csv(file_path, sep='\t')
        # Rimuovo la prima colonna siccome non la utilizzerò
        data = data[data.columns[1:]]
        
        return data
    except Exception as e:
        print(f'Error loading dataset: {e}')
        return []
    
file_path = './TLN-definitions-23.tsv'

defs_df = load_dataset(file_path)   
defs_df.head(4)


,door,ladybug,pain,blurriness
0,"A construction used to divide two rooms, tempo...","small flying insect, typically red with black ...",A feeling of physical or mental distress,sight out of focus
1,"It's an opening, it can be opened or closed.","It is an insect, it has wings, red with black ...","It is a feeling, physical or emotional. It is ...","It is the absence of definite borders, shapele..."
2,"An object that divide two room, closing an hol...",An insect that can fly. It has red or orange c...,A felling that couscious beings can experince ...,A sensation felt when you can't see clearly th...
3,Usable for access from one area to another,Small insect with a red back,Concept that describes a suffering living being,Lack of sharpness


### Preprocessing
La funzione `preprocess` viene utilizzata per preelaborare i dati di testo. Prende in input una frase ed esegue le seguenti operazioni:

1. Tokenizzazione: La frase viene suddivisa in singole parole.
2. Rimozione delle stopword: Ogni parola presente nell'elenco delle stopword viene rimossa dalla frase. Le stopword sono parole comuni che non hanno molto significato e vengono spesso rimosse nelle attività di elaborazione del linguaggio naturale.
3. Lemmatizzazione: Ogni parola della frase viene lemmatizzata. 
4. `Filtro` dei sinonimi WordNet: Tutte le parole che non hanno un corrispondente synset (insieme di sinonimi) in WordNet vengono eliminate. Questo perché utilizziamo WordNet per comprendere il contenuto semantico del testo, quindi ogni parola che non è presente in WordNet non ci è utile.

La funzione restituisce la frase preelaborata come insieme di parole (Bag Of Words)

In [10]:
import string
lemmatizer = WordNetLemmatizer()
additional_stopwords = ['\'s', '’']
punctuation = set(string.punctuation)
stopwordset = set(stopwords.words('english') + additional_stopwords)


def to_lower_case(words):
    return words.lower()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def lemmatize(words):
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word.lower(), pos='v')  # Specify the part-of-speech tag 'v' for verb
        lemmas.append(lemma)
    return lemmas


def remove_stopwords(words):
    return [word for word in words if word not in stopwordset]

def remove_punctuation(words):
    return [word for word in words if word not in punctuation]

def filter_contained_in_wd(words):
    return {x for x in words if len(wn.synsets(x)) > 0}

def preprocess_data(sentence):
    words = to_lower_case(sentence)
    words = tokenize(words)
    words = lemmatize(words)
    words = remove_stopwords(words)
    words = remove_punctuation(words)
    words = filter_contained_in_wd(words)
    return words
  

sentence1 = 'He went to the bank to deposit my money asda'
sentence2 = 'The river bank is full of wild flowers'

context1_bow = preprocess_data(sentence1)
context2_bow = preprocess_data(sentence2)
print("Bag of words for context 1: ", context1_bow)
print("Bag of words for context 2: ", context2_bow)


Bag of words for context 1:  {'go', 'deposit', 'money', 'bank'}
Bag of words for context 2:  {'flower', 'wild', 'bank', 'full', 'river'}


Utilizzando l'approccio Lesk-based per Word Sense Disambiguation (WSD), identifichiamo il senso corretto per i lemmi estratti dai generi candidati.

L'algoritmo Lesk si basa sulla ricerca dei contesti circostanti una parola all'interno di un dizionario come WordNet. Questo contesto viene utilizzato per determinare il significato più appropriato della parola all'interno del contesto specifico in cui viene utilizzata. 

Non è stato reimplementato l'algoritmo di Lesk ma utilizzo l'implementazione presente in nltk. Ho comunque caricato il file di esercitazione relativo alla Word Sense Disambiguation dove ho realizzato e descritto in dettaglio l'implementazione di Lesk per il laboratorio della seconda parte del corso.  

`nota` Relativo al task della Word Sense Disambiguation, utilizzeremo l'euristica della "frequenza" per identificare le word più importanti, selezionando solo queste per limitare la ricerca di sensi.


`nota 2` Relativo al task della Word Sense Inference, utilizzeremo l'euristica della "frequenza" per identificare anche i "Genus Candidates" quindi i sensi da preferire, perché più ricorrenti
nelle varie definizioni. 


In [11]:
from nltk.wsd import lesk


def calculate_word_frequencies(sentences):
   
    freqs = Counter()
    # For each sentence in the list, calculate the word frequencies and add them to freqs
    for sentence in sentences:
        words = preprocess_data(sentence)
        freqs += Counter(words)
    # Return freqs
    return freqs

def disambiguate_senses(sentence):
    """
    The function preprocesses the sentence into words, disambiguates the senses of the words using the Lesk algorithm,
    and returns a Counter object of the senses.
    """
    # Tokenize the sentence into words -> bag of words
    words = preprocess_data(sentence)
    # Disambiguate the senses of the words
    senses = [lesk(context_sentence=words, ambiguous_word=word) for word in words]
    # Remove None values
    senses = [sense for sense in senses if sense is not None]
    # Return a Counter object of the senses
    return Counter(senses)


sentences = ['He went to the bank to deposit my money',
                        'He went to the bank to open a new bank account',
                        'The river bank is full of wild flowers']
context = calculate_word_frequencies(sentences)
context= context.most_common(15)
#convert the context back to a Counter object
context = Counter(dict(context))
print("Context: ", context)
for i,sentence in enumerate(sentences):
    print("Senses for sentence",i,disambiguate_senses(sentence))




Context:  Counter({'bank': 3, 'go': 2, 'deposit': 1, 'money': 1, 'open': 1, 'new': 1, 'account': 1, 'flower': 1, 'wild': 1, 'full': 1, 'river': 1})
Senses for sentence 0 Counter({Synset('rifle.v.02'): 1, Synset('deposit.n.04'): 1, Synset('money.n.03'): 1, Synset('savings_bank.n.02'): 1})
Senses for sentence 1 Counter({Synset('unfold.v.04'): 1, Synset('newfangled.s.01'): 1, Synset('rifle.v.02'): 1, Synset('report.v.01'): 1, Synset('deposit.v.02'): 1})
Senses for sentence 2 Counter({Synset('flower.n.03'): 1, Synset('wilderness.n.03'): 1, Synset('deposit.v.02'): 1, Synset('entire.s.01'): 1, Synset('river.n.01'): 1})


Formalizziamo quanto descritto prima definendo la funzione `generate_genus_candidates`

In [12]:
def generate_genus_candidates(data, genus_n = 5, context_words =25 ):
    """
    This function explores WordNet to find the best sense for a list of sentences.
    The function disambiguates the senses of the words in the sentences and counts their frequencies, chooses candidate
    "genus" senses, 

    """
    # Disambiguation and frequency count, only for words in Wordnet
    context = calculate_word_frequencies(data)
    context= context.most_common(context_words)
    #convert the context back to a Counter object
    context = Counter(dict(context))
    candidate_senses = Counter()
    for sentence in data:
        candidate_senses += disambiguate_senses(sentence)
    # Choose candidate genus
    candidate_genus = Counter(dict(candidate_senses.most_common(genus_n)))

    return candidate_genus,context

sentences = ['He went to the bank to deposit my money asda',
                        'He created a new bank account',
                        'The river bank is full of flowers']

candidate_senses,context = generate_genus_candidates(sentences)
print("Merged Context : \n", context)

print("Genus Candidates : \n", candidate_senses)
# fix accesso .. nesting tremendo
print("\nTop 1 Genus Candidate definition: ", candidate_senses.most_common(1)[0][0].definition())


Merged Context : 
 Counter({'bank': 3, 'go': 1, 'deposit': 1, 'money': 1, 'create': 1, 'new': 1, 'account': 1, 'full': 1, 'flower': 1, 'river': 1})
Genus Candidates : 
 Counter({Synset('deposit.v.02'): 2, Synset('rifle.v.02'): 1, Synset('deposit.n.04'): 1, Synset('money.n.03'): 1, Synset('savings_bank.n.02'): 1})

Top 1 Genus Candidate definition:  put into a bank account


Esecuzione dell'algoritmo di Word Sense Induction
   - (a) Scelta dei candidati "genus" (selezionando i sensi più frequenti).
   - (b) Per ogni candidato "genus", esecuzione di una ricerca in profondità a partire dal sotto-albero del senso "genus" che massimizza la similarità con le definizioni.
   - (c) Scelta del senso con la massima similarità tra quelli estratti dai vari sotto-alberi dei candidati "genus".


All'interno della funzione wsi(), viene chiamata la funzione generate_genus_candidates(data, k) per ottenere i candidati "genus" più frequenti dal contesto dei dati. Successivamente, per ogni candidato "genus", viene calcolata la similarità con il contesto utilizzando la funzione calc_overlap(sense, context). Infine, i sensi candidati vengono ordinati in base alla similarità e i primi n sensi con la massima similarità vengono restituiti.


La similarità coseno è utilizzata nel calcolo dell'overlap tra i vettori di conteggio delle parole dei sensi e del contesto. Funziona efficacemente in questo contesto perché rappresenta una misura di similarità tra i vettori che tiene conto della direzione e della magnitudine dei vettori stessi.

In [13]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize


def sense_sig_with_freq(sense, levels = 1):
    """
    This function calculates the signature of a sense. 
    """

    # Calculate the signature from the definition of the sense + examples + synonyms

    signature_sentences = [sense.definition()] + sense.examples()
    sig = calculate_word_frequencies(signature_sentences)

    # Add the synonyms of the sense
    synonyms = {lemmatizer.lemmatize(x.lower()) for x in sense.lemma_names() if not '_' in x}
    sig += Counter(synonyms)

    # Add the word frequencies from the hypernyms, hyponyms, and meronyms of the sense.
    # This is done just for the first level of hypernyms, hyponyms, and meronyms.
    if levels > 0:
        hypernym_sig, hyponym_sig, meronym_sig = Counter(), Counter(), Counter()

        for hypernym in sense.hypernyms() + sense.instance_hypernyms():
            hypernym_sig += sense_sig_with_freq(hypernym, levels-1)

        for hyponym in sense.hyponyms() + sense.instance_hyponyms():
            hyponym_sig += sense_sig_with_freq(hyponym, levels-1)

        for meronym in sense.part_meronyms() + sense.member_meronyms():
            meronym_sig += sense_sig_with_freq(meronym, levels-1)

        sig += hypernym_sig
        sig += hyponym_sig
        sig += meronym_sig

    # Return the signature
    return sig


def calc_overlap(sense, context,levels = 1):

    # Get the signature of the sense
    sig = sense_sig_with_freq(sense, levels)
    # Convert the signature and the context to lists of words
    sig_words = list(sig.elements())
    context_words = list(context.elements())

    sig_string = ' '.join(sig_words)
    context_string = ' '.join(context_words)

    vectorizer = CountVectorizer().fit([sig_string, context_string])

    sig_vector = vectorizer.transform([sig_string]).toarray()
    context_vector = vectorizer.transform([context_string]).toarray()

    sig_vector = normalize(sig_vector)
    context_vector = normalize(context_vector)

    # Calculate the cosine similarity between the vectors
    similarity = cosine_similarity(sig_vector, context_vector)[0][0]
    # Return the similarity
    return similarity

def wsi(data,n=5, k =25,l = 1):
    '''
        This function performs Word Sense Induction (WSI).

        The function takes a list of candidate senses and a context as input, calculates the overlap    between each candidate sense
        and the context using the calc_overlap function, and returns the candidate sense with the   maximum overlap.
    '''

    candidate_genus,context = generate_genus_candidates(data,n,k)
    print(candidate_genus)
    sense_similarities = []
    for sense in candidate_genus:
        similarity = calc_overlap(sense, context,l)
        sense_similarities.append((sense, similarity))
    # Sort the senses by similarity in descending order and return the top n
    sense_similarities.sort(key=lambda x: x[1], reverse=True)

    sense_similarities = [(x[0].name(), x[1], x[0].definition()) for x in sense_similarities]

    return sense_similarities[:n]

file_path = './TLN-definitions-23.tsv'

data = load_dataset(file_path)   

data = defs_df['ladybug'].values.tolist()

best_senses = wsi(data,15,25,4)
print(best_senses)



Counter({Synset('red.n.01'): 26, Synset('blacken.v.01'): 21, Synset('small.s.08'): 20, Synset('insect.n.01'): 19, Synset('point.n.09'): 13, Synset('fly.v.13'): 11, Synset('worm.n.02'): 9, Synset('luck.n.03'): 8, Synset('coloring_material.n.01'): 7, Synset('normally.r.01'): 7, Synset('well.r.01'): 6, Synset('spot.v.06'): 5, Synset('dot.v.04'): 5, Synset('back.v.10'): 5, Synset('qualify.v.06'): 5})
[('insect.n.01', 0.45241571043344786, 'small air-breathing arthropod'), ('red.n.01', 0.3112711298872557, 'red color or pigment; the chromatic color resembling the hue of blood'), ('small.s.08', 0.2879627745785982, 'have fine or very small constituent particles'), ('coloring_material.n.01', 0.11049728283216656, 'any material used for its color'), ('point.n.09', 0.08010206801937156, 'a very small circular shape'), ('normally.r.01', 0.07580202540303349, 'under normal conditions'), ('qualify.v.06', 0.04244013201570598, 'describe or portray the character or the qualities or peculiarities of'), ('lu

### Visualizzazione

Visualizziamo i risultati ottenuti.
Impostando il parametro l=4 considero una espansione dell'albero su 4 livelli.




In [14]:
file_path = './TLN-definitions-23.tsv'

def format_results(results):
    output = ""
    for i, result in enumerate(results, 1):
        output += f"{i}. {result[0]}; score {result[1]:.4f}, distance {result[2]}\n"
    return output

data = load_dataset(file_path)   

output = ""
for column in defs_df.columns:
    data = defs_df[column].values.tolist()
    best_senses =wsi(data,15,25,4)[:5]
    output += f"\nConcept: {column}\n"
    output += format_results(best_senses)

print(output)

Counter({Synset('room.n.04'): 14, Synset('object.v.02'): 14, Synset('unfold.v.04'): 10, Synset('access.v.02'): 9, Synset('give_up.v.11'): 7, Synset('conclude.v.04'): 6, Synset('use.v.03'): 6, Synset('deuce.n.04'): 4, Synset('passing.n.03'): 4, Synset('wall.v.01'): 4, Synset('obstruct.v.02'): 4, Synset('lock.v.08'): 4, Synset('normally.r.01'): 4, Synset('entrance.v.02'): 4, Synset('record.v.01'): 3})
Counter({Synset('red.n.01'): 26, Synset('blacken.v.01'): 21, Synset('small.s.08'): 20, Synset('insect.n.01'): 19, Synset('point.n.09'): 13, Synset('fly.v.13'): 11, Synset('worm.n.02'): 9, Synset('luck.n.03'): 8, Synset('coloring_material.n.01'): 7, Synset('normally.r.01'): 7, Synset('well.r.01'): 6, Synset('spot.v.06'): 5, Synset('dot.v.04'): 5, Synset('back.v.10'): 5, Synset('qualify.v.06'): 5})
Counter({Synset('feel.v.08'): 13, Synset('forcible.s.01'): 12, Synset('emotional.a.03'): 10, Synset('sense.n.03'): 10, Synset('discomfort.n.02'): 5, Synset('unpleasant.a.01'): 5, Synset('induce.v.0

## Risultati

Attraverso un'analisi qualitativa, è emerso che l'algoritmo utilizzato non è in grado di individuare correttamente il senso desiderato. Un esempio diretto è il caso di "Door" che viene
associato a qualche senso legato al gioco del baseball, mentre uno dei significati o almeno contesto semantico simile è rappresentato da "obstruct.v.02" (anche se la definizione del synset di Wordnet non è molto chiara).

Alcune possibili strategie per migliorare le inferenze nel contesto dell'inferenza del senso delle parole:

- Ampliamento del contesto: Considerare contesti più ampi potrebbe consentire di ottenere una visione più completa delle parole e dei loro significati. Ad esempio, oltre alle definizioni, potrebbero essere inclusi anche esempi, sinonimi o altri contesti associati.

- Considerazione delle relazioni semantiche: Sfruttare le relazioni semantiche tra i sensi delle parole potrebbe essere utile per guidare le inferenze. Ad esempio, invece di considerare solo i sensi più frequenti, potrebbe essere valutato il grado di relazione con il concetto target, come iponimi o iperonimi.

- Utilizzo di risorse aggiuntive: Integrare altre risorse linguistiche, come corpus di testi o basi di conoscenza semantiche diverse da WordNet, potrebbe arricchire le informazioni disponibili per le inferenze. Ad esempio, l'utilizzo di ConceptNet potrebbe fornire una prospettiva più contestuale e associativa.


WordNet, pur essendo una preziosa risorsa per la conoscenza semantico-lessicale, potrebbe risultare limitata per questo specifico compito. Tra i principali limiti:
- Complessità computazionale: L'albero di WordNet è ampio e profondo, con numerosi sensi e relazioni tra di loro. Attraversarlo completamente richiederebbe molto tempo e risorse computazionali, rendendo l'approccio inefficiente per il WSI. Una dimostrazione è il fatto che solo impostando a 4 la profondità di ricerca, i tempi di elaborazione sono aumentati considerevolmente.

- Ambiguità dei sensi: Molti sensi in WordNet sono ambigui e possono avere molteplici interpretazioni all'interno di contesti diversi. Attraversare l'albero potrebbe portare a considerare sensi che non sono pertinenti al contesto specifico in cui si sta cercando di inferire il senso corretto.

- Scarsa rappresentazione delle relazioni semantiche: WordNet rappresenta principalmente relazioni iponimiche (gerarchiche) tra sensi, ma può mancare di altre relazioni semantiche importanti, come le relazioni associative o contestuali. Queste relazioni possono essere cruciali per la corretta inferenza del senso delle parole in un determinato contesto.

- Variazione tra le definizioni dei sensi: Le definizioni fornite in WordNet possono variare notevolmente in termini di stile, contenuto e completezza. Alcune definizioni potrebbero essere più descrittive e utili per il WSI, mentre altre potrebbero essere vaghe o poco informative.